In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from huggingface_hub import login


In [ ]:
login(token='')

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained("google/gemma-2-2b", use_auth_token=True)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


model-00001-of-00003.safetensors:   7%|6         | 336M/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/481M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.cuda.empty_cache()  # Clears any cached GPU memory

# Move model to the specified device
model.to(device)


Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2SdpaAttention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
          (rotary_emb): Gemma2RotaryEmbedding()
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), 

In [ ]:
tokenizer.padding_side = "left"

def generate_responses(json_data, max_length=20, batch_size=5):
    responses = []
    tokenizer.pad_token = tokenizer.eos_token

    # Prepare queries with context and additional instructions
    queries_with_context = [
        (
            f"Answer the following in strict format as: {item['Response Formats']}.\n"
            f"Question: {item['Query']}\n"
            f"Context: {item['Context']}\n"
            f"Additional Instructions: {item.get('Additional Instructions', '')}"
        )
        for item in json_data
    ]

    for i in range(0, len(queries_with_context), batch_size):
        batch_questions = queries_with_context[i:i + batch_size]

        # Tokenize inputs with left padding for generation
        model_input = tokenizer(batch_questions, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

        # Initialize generated_text with the input sequence
        generated_text = model_input['input_ids']
        attention_mask = model_input['attention_mask']

        # Generate tokens iteratively
        for _ in range(max_length):
            model_output = model(input_ids=generated_text, attention_mask=attention_mask)
            next_tokens = model_output.logits[:, -1:, :].argmax(dim=-1)

            # Update generated_text and attention_mask with the new token
            generated_text = torch.cat([generated_text, next_tokens], dim=-1)
            attention_mask = torch.cat([attention_mask, torch.ones((attention_mask.size(0), 1), device=device)], dim=-1)

        # Decode responses
        for gen_text, item in zip(generated_text, json_data[i:i + batch_size]):
            decoded_output = tokenizer.decode(gen_text, skip_special_tokens=True)

            # Append the answer to responses
            responses.append({
                "query": item['Query'],
                "response": answer
            })

    return responses


In [ ]:
json_file = "./ValueTest.json"
df = pd.read_json(json_file)
json_data = df.to_dict(orient='records')

# Generate responses and save
output_data = generate_responses(json_data)

with open('generated_responses.json', 'w') as f:
    json.dump(output_data, f, indent=2)

print("Responses have been saved to 'generated_responses.json'")


OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 11.06 MiB is free. Process 128225 has 14.73 GiB memory in use. Of the allocated memory 14.33 GiB is allocated by PyTorch, and 291.70 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)